## CLEANING ZONING BOARD OF APPEAL TRACKER DATA
This notebook contains the cleaning process of the zba data. Exploratory cleaning analysis (can be found in repository history) is removed and replaced with comment or markdown explanations to aid readability. Column descriptions are documented in the data_insights document.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import re
import Levenshtein
# (pip install python-Levenshtein)

pd.set_option('display.max_columns', 100)

# from google.colab import drive
# drive.mount('/content/drive')
# directory = '/content/drive/MyDrive/City of Boston: Permitting D/Project Files/data/zba.csv'
directory = '../data/raw_zba.csv' # interchangeable with above code

# Estimated runtime ~1 minute

In [2]:
df = pd.read_csv(directory)
df.head()

,address,status,parent_apno,boa_apno,appeal_type,contact,submitted_date,received_date,hearing_date,ever_deferred,num_deferrals,final_decision_date,decision,closed_date,city,zip,ward,zoning_district,project_description
0,27 Hopkins Rd Jamaica Plain 02130,Appeal Submitted,NaN,BOA1534017,Zoning,Anthony Ross,2023-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jamaica Plain,2130,19,Jamaica Plain Neighborhood,NaN
1,3927 Washington ST Roslindale 02131,Appeal Submitted,NaN,BOA1534015,Zoning,Anthony Ross,2023-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Roslindale,2131,19,Jamaica Plain Neighborhood,NaN
2,206 Byron ST East Boston 02128,Community Process,ALT1486864,BOA1533753,Zoning,Marc LaCasse,2023-10-06,2023-10-06,NaN,NaN,NaN,NaN,NaN,NaN,East Boston,2128,1,East Boston Neighborhood,Change Occupancy 2 family to 3 Extend living s...
3,152 Washington ST Brighton 02135,Community Process,ALT1527237,BOA1533784,Zoning,Tinh Le,2023-10-06,2023-10-06,NaN,NaN,NaN,NaN,NaN,NaN,Brighton,2135,21,Allston/Brighton Neighborhood,change Occupancy from 2 family to 3 add bathro...
4,29 Orange St Roslindale 02131,Community Process,ERT1515294,BOA1533227,Zoning,Ivan Hernandez,2023-10-05,2023-10-05,NaN,NaN,NaN,NaN,NaN,NaN,Roslindale,2131,20,Roslindale Neighborhood,Building new 2 family duplex home Market rate ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9040 entries, 0 to 9039
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address              9040 non-null   object 
 1   status               9040 non-null   object 
 2   parent_apno          8771 non-null   object 
 3   boa_apno             9040 non-null   object 
 4   appeal_type          9040 non-null   object 
 5   contact              9040 non-null   object 
 6   submitted_date       9040 non-null   object 
 7   received_date        8516 non-null   object 
 8   hearing_date         7874 non-null   object 
 9   ever_deferred        7874 non-null   object 
 10  num_deferrals        7874 non-null   float64
 11  final_decision_date  8289 non-null   object 
 12  decision             8293 non-null   object 
 13  closed_date          8311 non-null   object 
 14  city                 9040 non-null   object 
 15  zip                  9040 non-null   i

The following contains the dropping of columns and cleaning of remaining columns, we're going to be consistent with lowercase underscored variable names

In [4]:
# Initial dropping columns
df.drop(columns=['address'], inplace=True)              # location data already available
df.drop(columns=['contact'], inplace=True)              # not useful for analysis
df.drop(columns=['ever_deferred'], inplace=True)        # can be determined with num_deferrals

In [5]:
# Initial renaming columns
df = df.rename(columns={'project_description': 'text'})
df = df.rename(columns={'num_deferrals': 'deferrals'})
df = df.rename(columns={'zip': 'zipcode'})

We start by changing the status and appeal_type columns to sets of integers

In [6]:
status_to_number = {
    'Appeal Submitted': 1,
    'Community Process': 2,
    'Hearing Scheduled': 3,
    'Hearing Rescheduled': 4,
    'Hearing Concluded': 5,
    'ZBA Decision Finalized': 6,
    'Appeal Closed': 7
}

df.status = df.status.map(status_to_number)

appeal_type_to_num = {'Zoning':0, 'Building':1}

df.appeal_type = df.appeal_type.map(appeal_type_to_num)

We can also remove the prefixes from the apno columns (note this difference in the inital and concluding df.head() calls)

In [7]:
df['parent_apno'] = df['parent_apno'].apply(lambda x: ''.join(filter(str.isdigit, str(x))) if not pd.isna(x) else np.nan)
df['boa_apno'] = df['boa_apno'].apply(lambda x: ''.join(filter(str.isdigit, str(x))) if not pd.isna(x) else np.nan)

Next we change dates (we only consider the issued_date for now because of its lack of nulls and resulting usefulness as the primary time series). Instead of keeping the other dates we will calculate durations in days between the submission date and the other checkpoints in the process.

In [8]:
# Extract time data from dates and convert other date_columns in preparation for the next step
df['submitted_date'] = pd.to_datetime(df['submitted_date'])
df = df.assign(year=df['submitted_date'].dt.year.astype('Int64'),
               month=df['submitted_date'].dt.month.astype('Int64'),
               day=df['submitted_date'].dt.day.astype('Int64'))

df['hearing_date'] = pd.to_datetime(df['hearing_date'], errors='coerce')
df['hearing_date'] = pd.to_datetime(df['hearing_date'], errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], errors='coerce')
df['final_decision_date'] = pd.to_datetime(df['final_decision_date'], errors='coerce')

# Get durations
def calculate_duration(row, col, col2):
    if pd.isna(row[col]) or pd.isna(row[col2]):
        # return pd.Timedelta(days=0)
        return pd.NaT
    else:
        return row[col] - row[col2]

df['hearing_submitted'] = df.apply(lambda x: calculate_duration(x, 'hearing_date', 'submitted_date'), axis=1)
df['hearing_submitted'] = df['hearing_submitted'].dt.days

df['final_submitted'] = df.apply(lambda x: calculate_duration(x, 'final_decision_date', 'submitted_date'), axis=1)
df['final_submitted'] = df['final_submitted'].dt.days

df['final_hearing'] = df.apply(lambda x: calculate_duration(x, 'final_decision_date', 'hearing_date', ), axis=1)
df['final_hearing'] = df['final_hearing'].dt.days

df['closed_submitted'] = df.apply(lambda x: calculate_duration(x, 'closed_date', 'submitted_date'), axis=1)
df['closed_submitted'] = df['closed_submitted'].dt.days

df['closed_final'] = df.apply(lambda x: calculate_duration(x, 'closed_date', 'final_decision_date'), axis=1)
df['closed_final'] = df['closed_final'].dt.days

df['closed_hearing'] = df.apply(lambda x: calculate_duration(x, 'closed_date', 'hearing_date'), axis=1)
df['closed_hearing'] = df['closed_hearing'].dt.days

# Drop columns
df.drop(columns=['submitted_date'], inplace=True)
df.drop(columns=['received_date'], inplace=True)
df.drop(columns=['hearing_date'], inplace=True)
df.drop(columns=['closed_date'], inplace=True)
df.drop(columns=['final_decision_date'], inplace=True) 

Since there were extra entry types for the decision column, we made sure there were only three options: Approved, Denied and Withdrawn, and handled empty strings

In [9]:
df['decision'] = df['decision'].replace({'AppProv': 'Approved', 'DeniedPrej': 'Denied'})
df.loc[df['decision'] == ' ', 'decision'] = np.nan

Removed Neighborhood with ' ' in the zoning_district field

In [10]:
df['zoning_district'] = df['zoning_district'].str.replace(' Neighborhood', '', regex=False)
df.zoning_district.value_counts()

zoning_district
Dorchester                                              1514
South Boston                                            1114
East Boston                                              912
Roxbury                                                  673
Allston/Brighton                                         573
Jamaica Plain                                            566
South End                                                483
Boston Proper                                            469
West Roxbury                                             430
Roslindale                                               389
Charlestown                                              388
Hyde Park                                                377
Greater Mattapan                                         370
North End                                                185
Mission Hill                                             102
Fenway                                                    93
Chinatow

Cities needed more extensive processing, we clean the data and levenshtein-match it

In [11]:
expected_cities = ["Boston", "West Roxbury", "Charlestown", "Roxbury", "Brighton", "Allston", "Jamaica Plain", "East Boston",
                   "Dorchester", "Hyde Park", "South Boston", "Roslindale", "Brighton/Allston", "Mission Hill", "Mattapan",
                   "Longwood", "Bay Village", "Chestnut Hill", "North End", "Leather District", "Chinatown",
                   "South Boston Waterfront", "West End", "Fenway", "South End", "Back Bay", "Downtown", "Beacon Hill",
                   "Theater District"]

def clean_and_match(city):
    if pd.isna(city):
        return city
    cleaned_city = ''.join(filter(str.isalpha, city))
    closest_match = min(expected_cities, key=lambda x: Levenshtein.distance(cleaned_city.lower(), x.lower()))
    return closest_match

df['city'] = df['city'].apply(clean_and_match)

We will also reuse the zipcode cleaning (although here we mostly only need to add 0s)

In [12]:
def clean_zip(value):
    if pd.isna(value):
        return value

    value = str(value)
    value = re.sub(r'-.*|\.0$', '', value)
    value = '0' + value if len(value) == 4 else value
    value = pd.NA if value.isdigit() and len(value) <= 3 else value

    return value

df['zipcode'] = df['zipcode'].apply(clean_zip)

Preliminary cleaning/processing of the text data

In [13]:
def process_string(input_string):
    only_alphabetical = re.sub(r'[^a-zA-Z\s]', '', str(input_string))
    lowercased = only_alphabetical.lower()
    return lowercased

df.text = df.text.apply(process_string)

Now let's do a quick sanity check and write to csv

In [14]:
df.sample(5)

,status,parent_apno,boa_apno,appeal_type,deferrals,decision,city,zipcode,ward,zoning_district,text,year,month,day,hearing_submitted,final_submitted,final_hearing,closed_submitted,closed_final,closed_hearing
8963,7,272757,278615,0,NaN,Approved,West Roxbury,02132,20,West Roxbury,build new rear addition renovate kitchen dinin...,2013,8,23,NaN,180.0,NaN,200.0,20.0,NaN
6086,7,667510,672071,0,NaN,Withdrawn,Roslindale,02131,19,Roslindale,erection eight residential units two attached ...,2017,1,11,NaN,1696.0,NaN,1696.0,0.0,NaN
4184,7,49865826,893699,0,0.0,Approved,South Boston,02127,6,South Boston,create four accessory residential off street p...,2018,10,25,82.0,99.0,17.0,99.0,0.0,17.0
5568,7,711604,745475,0,0.0,Approved,Roslindale,02131,20,Roslindale,install new sliding door into bedroom exitting...,2017,8,16,314.0,317.0,3.0,317.0,0.0,3.0
431,7,1424845,1446580,0,0.0,Approved,Mattapan,02126,18,Greater Mattapan,confirm occupancy as family dwelling this is ...,2023,3,7,77.0,136.0,59.0,157.0,21.0,80.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9040 entries, 0 to 9039
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   status             9040 non-null   int64  
 1   parent_apno        8771 non-null   object 
 2   boa_apno           9040 non-null   object 
 3   appeal_type        9040 non-null   int64  
 4   deferrals          7874 non-null   float64
 5   decision           8289 non-null   object 
 6   city               9040 non-null   object 
 7   zipcode            9040 non-null   object 
 8   ward               9040 non-null   int64  
 9   zoning_district    9000 non-null   object 
 10  text               9040 non-null   object 
 11  year               9040 non-null   Int64  
 12  month              9040 non-null   Int64  
 13  day                9040 non-null   Int64  
 14  hearing_submitted  7874 non-null   float64
 15  final_submitted    8289 non-null   float64
 16  final_hearing      7621 

## SAVE CSV

In [16]:
# df.to_csv('/content/drive/MyDrive/City of Boston: Permitting D/Project Files/data/zba_cleaned.csv', index=False, encoding='utf-8')
df.to_csv('../data/cleaned_zba.csv', index=False, encoding='utf-8')